In [1]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
import re

In [2]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.read_pdf('condo.pdf', pages='all',lattice=True)

In [3]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [4]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [5]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [6]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

In [7]:
df['loc']

1     (125 Ocean Dr, Miami Beach, FL 33139, USA, (25...
2     (6899 Collins Ave, Miami Beach, FL 33141, USA,...
3     (1300 Monad Terrace, Miami Beach, FL 33139, US...
4     (4701 Meridian Ave, Miami Beach, FL 33140, USA...
5     (10 Venetian Way, Miami Beach, FL 33139, USA, ...
6     (3000 Island Blvd PH-4, Aventura, FL 33160, US...
7     (3737 Collins Ave, Miami Beach, FL 33140, USA,...
8     (10101 Collins Ave #12a, Bal Harbour, FL 33154...
9     (16 Island Ave #7b, Miami Beach, FL 33139, USA...
10    (900 Brickell Key Blvd #402, Miami, FL 33131, ...
Name: loc, dtype: object

In [8]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [9]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Buyer Agent,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,Ocean House\r125 Ocean Dr U-0503\rMiami Beach,10/15/2021,1/12/2022,38,"$9,550,000.00",5058,"$2,561.70",Dora Puig,Judith Werner,Ocean House 125 Ocean Dr U-0503 Miami Beach,Ocean House,125 Ocean Dr U-0503 Miami Beach,"(125 Ocean Dr, Miami Beach, FL 33139, USA, (25...","(25.7702756, -80.13276069999999, 0.0)",25.770276,-80.132761,0.0
2,North Carillon Beach Condo\r6899 Collins Ave 3...,11/12/2021,1/10/2022,39,"$3,200,000.00",2436,"$1,313.63",Kayce Driscoll,Tania De Jesus PA,North Carillon Beach Condo 6899 Collins Ave 31...,North Carillon Beach Condo,6899 Collins Ave 3101 Miami Beach,"(6899 Collins Ave, Miami Beach, FL 33141, USA,...","(25.85381, -80.11977000000002, 0.0)",25.853810,-80.119770,0.0
3,Monad Terrace\r1300 Monad Ter 10E\rMiami Beach,10/25/2021,1/14/2022,21,"$3,175,000.00",5465,$759.39,Ann Nortmann PA,Esther Percal,Monad Terrace 1300 Monad Ter 10E Miami Beach,Monad Terrace,1300 Monad Ter 10E Miami Beach,"(1300 Monad Terrace, Miami Beach, FL 33139, US...","(25.7840774, -80.1425416, 0.0)",25.784077,-80.142542,0.0
4,The Residences at Miami Beach\r4701 N Meridian...,10/18/2021,1/14/2022,36,"$3,000,000.00",2043,"$1,468.43",Lucas Boccheciampe,Geane Brito,The Residences at Miami Beach 4701 N Meridian ...,The Residences at Miami Beach,4701 N Meridian Ave 322 Miami Beach,"(4701 Meridian Ave, Miami Beach, FL 33140, USA...","(25.8230958, -80.1312295, 0.0)",25.823096,-80.131230,0.0
5,The Grand Venetian\r10 Venetian Way 1704\rMiam...,10/18/2021,1/12/2022,69,"$2,975,000.00",2345,"$1,268.66",Betsy Magde,Ben Giordano,The Grand Venetian 10 Venetian Way 1704 Miami ...,The Grand Venetian,10 Venetian Way 1704 Miami Beach,"(10 Venetian Way, Miami Beach, FL 33139, USA, ...","(25.7911488, -80.1468276, 0.0)",25.791149,-80.146828,0.0
6,Williams Island\r3000 Island Blvd PH-4\rAventura,1/1/2022,1/13/2022,6,"$2,850,000.00",4232,$673.44,Joel Matus,Celia Mussman,Williams Island 3000 Island Blvd PH-4 Aventura,Williams Island,3000 Island Blvd PH-4 Aventura,"(3000 Island Blvd PH-4, Aventura, FL 33160, US...","(25.9406051, -80.1344671, 0.0)",25.940605,-80.134467,0.0
7,The Caribbean Condo\r3737 Collins Ave S-1403\r...,9/13/2021,1/14/2022,93,"$2,650,000.00",1621,"$1,634.79",Ida Schwartz,Minette Schwartz,The Caribbean Condo 3737 Collins Ave S-1403 Mi...,The Caribbean Condo,3737 Collins Ave S-1403 Miami Beach,"(3737 Collins Ave, Miami Beach, FL 33140, USA,...","(25.810647, -80.122714, 0.0)",25.810647,-80.122714,0.0
8,The Palace\r10101 Collins Ave 12A\rBal Harbour,11/24/2021,1/13/2022,35,"$2,450,000.00",3110,$787.78,Allen Davoudpour,Joelle Oiknine,The Palace 10101 Collins Ave 12A Bal Harbour,The Palace,10101 Collins Ave 12A Bal Harbour,"(10101 Collins Ave #12a, Bal Harbour, FL 33154...","(25.8926977, -80.1228812, 0.0)",25.892698,-80.122881,0.0
9,Belle Towers Condo\r16 Island Ave 7B\rMiami Beach,12/8/2021,1/14/2022,1,"$2,000,000.00",1770,"$1,129.94",Hillary Hertzberg,Elyse Rosenberg,Belle Towers Condo 16 Island Ave 7B Miami Beach,Belle Towers Condo,16 Island Ave 7B Miami Beach,"(16 Island Ave #7b, Miami Beach, FL 33139, USA...","(25.7903154, -80.147503, 0.0)",25.790315,-80.147503,0.0
10,Asia Condominium\r900 Brickell Key Blvd 703\rM...,8/7/2021,1/12/2022,104,"$1,900,000.00",2377,$799.33,Dany Perl,Martha Dajer,Asia Condominium 900 Brickell Key Blvd 703 Miami,Asia Condominium,900 Brickell Key Blvd 703 Miami,"(900 Brickell Key Blvd #402, Miami, FL 33131, ...","(25.7692204, -80.185768, 0.0)",25.769220,-80.185768,0.0


Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [10]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Buyer Agent',
       'geo_address', 'building_name', 'address_only', 'loc', 'point', 'lat',
       'lon', 'altitude'],
      dtype='object')

In [13]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=11)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('January 9th - January 15th')

marker_cluster = MarkerCluster().add_to(m)

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['Sq. Ft.'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Name: {bold_start}{info[-1]}{bold_end}{new_line} \
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Price sq ft: \
    {bold_start}${info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(marker_cluster)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [14]:
m.save('index.html')